## Profiling proseco / sparkles for impact of overlap prevention


In [1]:
import gzip
import pickle
import sys
from pathlib import Path

sys.path.insert(0, str(Path.home() / "git" / "proseco"))

In [2]:
import ska_helpers.utils

In [3]:
import proseco

print(proseco.test(get_version=True))
print(proseco.__file__)

5.12.1.dev2+g127316a.d20240309-r938-127316a
/Users/aldcroft/git/proseco/proseco/__init__.py


In [4]:
import sparkles

print(sparkles.test(get_version=True))
print(sparkles.__file__)

4.26.0
/Users/aldcroft/miniconda3/envs/ska3-flight-2024.1rc4/lib/python3.11/site-packages/sparkles/__init__.py


In [5]:
filename = "/Users/aldcroft/ska/data/mpcrit1/mplogs/2022/NOV2122/oflsa/output/NOV2122A_proseco.pkl.gz"
acas_dict = pickle.load(gzip.open(filename, "rb"))

In [6]:
def run_one(aca, run_proseco=True, run_sparkles=True, overlap_penalty=False):
    with ska_helpers.utils.temp_env_var(
        "PROSECO_DISABLE_OVERLAP_PENALTY", f"{not overlap_penalty}"
    ):
        if run_proseco:
            aca = proseco.get_aca_catalog(**aca.call_args)
        if run_sparkles:
            acar = aca.get_review_table()
            acar.run_aca_review()

In [7]:
def run_all(acas_dict, run_proseco=True, run_sparkles=True, overlap_penalty=False):
    for aca in acas_dict.values():
        run_one(
            aca,
            run_proseco=run_proseco,
            run_sparkles=run_sparkles,
            overlap_penalty=overlap_penalty,
        )

In [8]:
run_one(acas_dict[27565])

In [9]:
%prun -T tmp.txt -l 20 -s cumulative run_all(acas_dict, run_sparkles=False, overlap_penalty=False)

 
*** Profile printout saved to text file 'tmp.txt'.


         24467498 function calls (23591548 primitive calls) in 13.462 seconds

   Ordered by: cumulative time
   List reduced from 1409 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    133/1    0.000    0.000   13.463   13.463 {built-in method builtins.exec}
        1    0.009    0.009   13.463   13.463 1492753284.py:1(run_all)
       37    0.001    0.000   13.455    0.364 1317233785.py:1(run_one)
       37    0.000    0.000   13.453    0.364 __init__.py:6(get_aca_catalog)
       37    0.000    0.000   13.453    0.364 catalog.py:47(get_aca_catalog)
       37    0.004    0.000   13.453    0.364 catalog.py:82(_get_aca_catalog)
       37    0.004    0.000    8.273    0.224 acq.py:180(get_acq_catalog)
      709    0.081    0.000    5.326    0.008 acq.py:1540(__init__)
    14889    0.179    0.000    3.226    0.000 acq.py:880(calc_p_brightest)
       37    0.002    0.000    3.194    0.086 guide.py:33(get_guide_catalog)
       37    0.

In [10]:
%prun -T tmp.txt -l 20 -s cumulative run_all(acas_dict, run_sparkles=False, overlap_penalty=True)

 
*** Profile printout saved to text file 'tmp.txt'.


         44458835 function calls (42372674 primitive calls) in 19.380 seconds

   Ordered by: cumulative time
   List reduced from 1410 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    137/1    0.000    0.000   19.381   19.381 {built-in method builtins.exec}
        1    0.011    0.011   19.381   19.381 1492753284.py:1(run_all)
       37    0.000    0.000   19.370    0.524 1317233785.py:1(run_one)
       37    0.000    0.000   19.369    0.523 __init__.py:6(get_aca_catalog)
       37    0.000    0.000   19.368    0.523 catalog.py:47(get_aca_catalog)
       37    0.005    0.000   19.368    0.523 catalog.py:82(_get_aca_catalog)
       37    0.005    0.000   14.083    0.381 acq.py:180(get_acq_catalog)
       37    0.010    0.000    7.465    0.202 acq.py:1118(optimize_catalog)
      989    0.022    0.000    7.062    0.007 acq.py:1016(optimize_acq_halfw)
     5578    0.095    0.000    6.988    0.001 acq.py:966(calc_p_safe)
    33568  

In [11]:
17.9 / 12.9

1.387596899224806

In [12]:
%time run_one(acas_dict[27565])

CPU times: user 258 ms, sys: 3.84 ms, total: 262 ms
Wall time: 257 ms


In [13]:
acas_dict.keys()

dict_keys([27565.0, 27566.0, 25219.0, 27223.0, 27567.0, 27571.0, 26874.0, 27051.0, 26644.0, 45049.0, 45048.0, 45046.0, 45045.0, 27477.0, 25873.0, 25321.0, 27564.0, 27570.0, 27484.0, 26855.0, 26500.0, 45044.0, 45043.0, 45042.0, 45040.0, 45039.0, 45038.0, 26718.0, 27483.0, 25963.0, 27568.0, 27522.0, 25938.0, 27569.0, 25937.0, 26870.0, 45037.0])

In [14]:
%time run_all(acas_dict, run_sparkles=False, overlap_penalty=False)

CPU times: user 7.62 s, sys: 81.5 ms, total: 7.7 s
Wall time: 7.6 s


In [15]:
%time run_all(acas_dict, run_sparkles=False, overlap_penalty=True)

CPU times: user 9.48 s, sys: 90 ms, total: 9.57 s
Wall time: 9.53 s


In [17]:
9.5 / 7.6

1.25